In [1]:
import re
import emoji
from soynlp.normalizer import repeat_normalize
from transformers import ElectraForSequenceClassification, ElectraTokenizer, ElectraConfig
import torch.nn as nn
import pandas as pd

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from fastprogress.fastprogress import master_bar, progress_bar
from attrdict import AttrDict

from finetune.processor import seq_cls_tasks_num_labels as tasks_num_labels
from finetune.processor import seq_cls_processors as processors
from finetune.processor import seq_cls_output_modes as output_modes

from finetune.processor.seq_cls import seq_cls_convert_examples_to_features
from finetune.processor.seq_cls import InputExample


import json
import torch
from torch.utils.data import TensorDataset
import numpy as np

In [2]:
def compute_metrics(task_name, labels, preds):
    assert len(preds) == len(labels)
    if task_name == "kornli":
        return acc_score(labels, preds)
    elif task_name == "nsmc":
        return acc_score(labels, preds)
    elif task_name == "paws":
        return acc_score(labels, preds)
    elif task_name == "korsts":
        return pearson_and_spearman(labels, preds)
    elif task_name == "question-pair":
        return acc_score(labels, preds)
    elif task_name == "naver-ner":
        return f1_pre_rec(labels, preds, is_ner=True)
    elif task_name == "hate-speech":
        return f1_pre_rec(labels, preds, is_ner=False)
    else:
        raise KeyError(task_name)
        
def acc_score(labels, preds):
    return {
        "acc": simple_accuracy(labels, preds),
    }
def simple_accuracy(labels, preds):
    return (labels == preds).mean()

In [3]:
lines = []

for i in range(len(train_df)):
    line = train_df.iloc[i]
    line_review1 = line[['reviewId']].to_string(header=False,index=False).strip()
    line_review2 = line[['content']].to_string(header=False,index=False).strip()
    line_review3 = line[['score']].to_string(header=False,index=False).strip()
    
    lines.append(line_review1 + '\t' + line_review2 + '\t' + line_review3)

NameError: name 'train_df' is not defined

In [42]:
line = train_df.iloc[1]
line[['reviewId']].to_string(header=False,index=False).strip() + '11'

'0df5ae4e-ff08-434a-b7a2-62da3181b58611'

In [36]:
line[['reviewId', 'content', 'score']].to_string(header=False,index=False).strip()

'edd3bbc0-03e2-4a8a-aa32-7e8a7481d7e0\n예전에는 없었던 트위터로 뉴스 바로 내보내기. 원하는 메뉴만 즐겨 찾기 할 수 있는 ...\n                                                 1'

In [99]:
lines

['reviewId\tcontent\tscore',
 '4123d655-f7b7-4328-abd2-7d7dcfb1c21f\t직관성 떨어지고 오류나고 답답\t0',
 '39afd67f-746b-462b-a838-f8dd5efca23e\t이 부분 개선되면 참 좋겠습니다\t0',
 '01016f38-580c-4c64-a954-5abf1f796094\t불법이 아니라면 개인투자자의 투자자율성 편의를 강화하는게 앱의 활용성을 높이고 경쟁력을 높이는데 크게 기여할거 같습니다~~\t0',
 '2630586a-03c8-41bb-ac4e-04a961b0e01b\t칸이모자르면 차라리 등락금액을 빼고 거래량을 넣어주세요\t0',
 'c124f0ff-c50a-47dc-a971-d5cf2cb09696\t좋아요..👌.\t0',
 'fe21b9a7-1b83-4cdb-92fe-7dd9408bb1a2\t메모기능 추가하면 좋을것같아요\t0',
 '368d8582-f98d-47b0-870a-63ff475dc534\t이부분에 대해 고민해보시면 좋을거 같습니다\t0',
 'a70c790a-417d-4616-bf73-00f16ca97e8a\t글자가 안보임\t0',
 '05fe293e-9f2c-45b6-87ec-692def1196c8\t단지 모르는번호로 주식단타열락문자옴\t0',
 '0fe4c2b1-d5ce-4aab-adf7-b678564d5aec\t개심해요 주식현재가 차트 보다가 위에 검색창에서 검색할라하면은 갑자기 미리보기 안뜨고 뒤로 나가기 누르니 화면이 회색깔로 변하고 뭐하는 짓임?\t0',
 '834397cf-c407-47cf-8ec6-28e29f7c5fed\tQv계좌연동은 안됩니까\t0',
 'ed4ecb2f-1915-49f4-8942-ba68f6d92833\t특별한 업데이트도 없는거 같고..\t0',
 'f6985ffb-e5fd-4518-9b5d-2eaa88bae5a2\t다른 증권사앱들은 종목 뉴스탭에 제대로 필터링돼서 뉴스들을 보여주는데 나무는 매번 이상한 광고 찌라시 뉴스만 잔뜩있어서 

In [21]:
line

'9619869\t한국 영화 최초로 수간하는 내용이 담긴 영화\t0\n'

In [20]:
lines.replace('\n', '\t')

AttributeError: 'list' object has no attribute 'replace'

In [101]:
with open('/home/hjgp/research/KoELECTRA/finetune/data/nsmc/namu_test.txt', "r", encoding="utf-8") as f:
    lines = []
    for line in f:
        lines.append(line.strip())


In [98]:
lines[-10:]

['98a949ac-bd90-476d-82c3-60a951d1cab0\t타사는 스마트거래시 수수료싸든데 여기도치별점\t0',
 'cf10f289-0a43-436a-af4d-e3323d1459cd\tㅋㅋ\t0',
 '36c2d9c6-b896-48f0-8de8-e73b4065651c\t부탁드려요!\t0',
 '279e70fb-4aff-429a-9b58-e59dd9927224\t주식초보자인데 직관적 UI가 편해요\t0',
 'ac780c2d-351a-42fb-be64-ef298cab823d\t처음엔 좋았는데 업글되더니 형편없음.\t0',
 '4c5a5486-9f87-4571-a97e-ba5353cddccc\t아쉬운건 수수료가 비싸다는 점..\t0',
 '9100e8a6-7d71-4ea7-a972-145d5c1d1821\t상당한 발전이군요\t0',
 'e620e368-649a-4d0d-a772-c4cab76470e1\t시세알림 기능도 유용합니다.\t0',
 '471a8ecd-a646-4435-8246-3c924edd647b\t로그인없이 시세 조회도 가능하고  일단 속도가 다른 어플에 비해서 빠르네요^^\t0',
 'edd3bbc0-03e2-4a8a-aa32-7e8a7481d7e0\t부탁드립니다.\t0']

In [102]:
class NsmcProcessor(object):
    """Processor for the NSMC data set """

    def __init__(self, args, df):
        self.args = args

    def get_labels(self):
        return ["0", "1"]

    @classmethod
    def _read_file(cls, df):
        """Reads a tab separated value file."""
        lines = df
#         lines = []

#         for i in range(len(train_df)):
#             line = train_df.iloc[i]
#             line_review1 = line[['reviewId']].to_string(header=False,index=False).strip()
#             line_review2 = line[['content']].to_string(header=False,index=False).strip()
#             line_review3 = line[['score']].to_string(header=False,index=False).strip()

#             lines.append(line_review1 + '\t' + line_review2 + '\t' + line_review3)
        return lines
    
    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines[1:]):
            line = line.split("\t")
            guid = "%s-%s" % (set_type, i)
            text_a = line[1].strip()
            label = line[2].strip()
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

    def get_examples(self, mode, df):
        """
        
        Args:
            mode: train, dev, test
        """
        file_to_read = None
        if mode == "train":
            file_to_read = self.args.train_file
        elif mode == "dev":
            file_to_read = self.args.dev_file
        elif mode == "test":
            file_to_read = self.args.test_file

        return self._create_examples(
            self._read_file(df), mode
        )

In [103]:
def infer(x, path) :
    model = ElectraForSequenceClassification.from_pretrained(path)
    tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")
    
    emojis = ''.join(emoji.UNICODE_EMOJI.keys())  
    pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
    url_pattern = re.compile(
        r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)'
    )
    processed = pattern.sub(' ', x)
    processed = url_pattern.sub(' ', processed)
    processed = processed.strip()
    processed = repeat_normalize(processed, num_repeats=2)

    tokenized = tokenizer(processed, return_tensors='pt')

    output = model(tokenized.input_ids, tokenized.attention_mask)
    return nn.functional.softmax(output.logits, dim=-1)

In [104]:
def seq_cls_load_and_cache_examples(args, tokenizer, mode, df):
    processor = NsmcProcessor(args, df)
    # Load data features from cache or dataset file

    examples = processor.get_examples("test", df)
    features = seq_cls_convert_examples_to_features(args, examples, tokenizer, max_length=args.max_seq_len, task=args.task)
    
    #torch_tensor_output = torch.tensor(df['output'].values)
    
    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
    return dataset

In [36]:
train_df = pd.read_csv('/home/hjgp/research/NH_intern/filtered_for_nsmc.csv')

In [12]:
train_df['score'] = train_df['score'].replace(1, 0)
train_df['score'] = train_df['score'].replace(2, 0)
train_df['score'] = train_df['score'].replace(4, 1)
train_df['score'] = train_df['score'].replace(5, 1)

In [13]:
train_df['content'] = train_df['content'].str.replace('\n', '')

In [14]:
# train_df.to_csv('/home/hjgp/research/KoELECTRA/finetune/data/nsmc/reviews_finetuning.txt', sep = '\t', index=False)

In [37]:
lines

['reviewId\tcontent\tscore',
 '4123d655-f7b7-4328-abd2-7d7dcfb1c21f\t직관성 떨어지고 오류나고 답답\t1',
 '39afd67f-746b-462b-a838-f8dd5efca23e\t이 부분 개선되면 참 좋겠습니다\t1',
 '8966822f-9b53-4aa5-b7f9-da2ad23f3740\t빠른 반영 부탁드립니다.\t3',
 '01016f38-580c-4c64-a954-5abf1f796094\t불법이 아니라면 개인투자자의 투자자율성 편의를 강화하는게 앱의 활용성을 높이고 경쟁력을 높이는데 크게 기여할거 같습니다~~\t1',
 '2630586a-03c8-41bb-ac4e-04a961b0e01b\t칸이모자르면 차라리 등락금액을 빼고 거래량을 넣어주세요\t0',
 '266d3498-87e9-4d62-939a-cb6ec8b59131\t왜 입금한 돈을 빼낼 수가 없는거죠??\t3',
 'c124f0ff-c50a-47dc-a971-d5cf2cb09696\t좋아요..👌.\t1',
 'fe21b9a7-1b83-4cdb-92fe-7dd9408bb1a2\t메모기능 추가하면 좋을것같아요\t1',
 '368d8582-f98d-47b0-870a-63ff475dc534\t이부분에 대해 고민해보시면 좋을거 같습니다\t1',
 'f2c30fb0-1c11-4fde-bfe0-f6d43753ab44\t뭐가 문제일까요?\t3',
 'a32b68b3-1dd3-4a18-8f46-de601682bd48\t한번 이런 로딩이 지속되면 며칠 간 들어갈 수가 없고 1시간씩 기다려도 접속이 안됩니다..\t3',
 'a70c790a-417d-4616-bf73-00f16ca97e8a\t글자가 안보임\t0',
 '64822085-362a-4c78-94de-ad912929ada7\t차트 확대 할때 가끔씩 보조지표까지 터치 할 경우 삭제 되는 경우가 많이 발생해서 일일이 다시 보조지표를 설정하는 상황이 발생합니다\t3',
 '05fe293e-9f2c-45b6-87

In [105]:
with open('/home/hjgp/research/KoELECTRA/finetune/config/nsmc/review_small_v3.json') as f:
    args = AttrDict(json.load(f))

processor = processors['nsmc'](args)

labels = processor.get_labels()
config = ElectraConfig.from_pretrained(
    '/home/hjgp/research/KoELECTRA/finetune/ckpt/koelectra-small-v3-nsmc-ckpt_review/checkpoint-21000/',
    num_labels=2,
    id2label={str(i): label for i, label in enumerate(labels)},
    label2id={label: i for i, label in enumerate(labels)},
)


tokenizer = ElectraTokenizer.from_pretrained(
    "monologg/koelectra-small-v3-discriminator",
)
model = ElectraForSequenceClassification.from_pretrained(
    '/home/hjgp/research/KoELECTRA/finetune/ckpt/koelectra-small-v3-nsmc-ckpt_review/checkpoint-21000/',
    config=config
)

test_dataset = seq_cls_load_and_cache_examples(args, tokenizer, mode="test", df=lines) ##train_df : 테스트하고싶은 df

In [106]:
def test(args, model, test_dataset):
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=100)

    preds = None

    for batch in progress_bar(test_dataloader):

        batch = tuple(t.to(args.device) for t in batch)
        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
            }

            outputs = model(**inputs)
            print(outputs[0])
            logits = outputs[0]

        if preds is None:
            preds = nn.functional.softmax(logits, dim=-1).detach().cpu().numpy()
        else:
            preds = np.append(preds, nn.functional.softmax(logits, dim=-1).detach().cpu().numpy(), axis=0)
        

#     result = compute_metrics(args.task, out_label_ids, preds)
#     results.update(result)

    return preds

In [107]:
args.device = "cuda"
result = test(args, model.cuda(), test_dataset)

tensor([[ 2.6556, -2.6765],
        [-0.7884,  0.7047],
        [-0.4074,  0.3922],
        [ 0.1247, -0.1522],
        [-2.0159,  1.8189],
        [-2.4535,  2.2069],
        [-2.4764,  2.2172],
        [ 1.6058, -1.6485],
        [-2.6069,  2.3893],
        [ 2.6348, -2.6330],
        [ 1.1423, -1.1602],
        [ 2.3962, -2.4542],
        [ 2.5948, -2.6348],
        [-2.6690,  2.4241],
        [ 2.3556, -2.3809],
        [-1.9883,  1.7956],
        [ 2.1653, -2.1898],
        [ 2.6042, -2.6099],
        [ 2.5295, -2.5435],
        [ 0.9803, -1.0096],
        [ 2.6792, -2.6951],
        [ 2.7780, -2.7729],
        [-2.0318,  1.8378],
        [-2.0250,  1.8294],
        [ 2.7879, -2.8294],
        [-2.2905,  2.0645],
        [ 2.6665, -2.6912],
        [ 0.8243, -0.8091],
        [-0.8808,  0.8058],
        [ 1.0996, -1.1250],
        [ 2.5988, -2.6242],
        [ 1.3892, -1.3463],
        [ 2.4590, -2.4459],
        [ 2.3794, -2.3640],
        [ 2.8221, -2.8340],
        [ 1.2825, -1

tensor([[-0.1852,  0.1738],
        [ 1.8856, -1.9055],
        [ 0.1270, -0.1906],
        [-1.3597,  1.1902],
        [ 2.5535, -2.5588],
        [ 1.3927, -1.4181],
        [ 2.6822, -2.7070],
        [ 0.0357, -0.0703],
        [ 1.0895, -1.1342],
        [-2.0325,  1.8310],
        [ 2.6096, -2.6131],
        [-2.1510,  1.9491],
        [ 2.9222, -2.9456],
        [-2.9103,  2.6079],
        [-2.5258,  2.2865],
        [ 2.6092, -2.6243],
        [ 0.6016, -0.5933],
        [ 2.6287, -2.6667],
        [ 2.9149, -2.9405],
        [-1.0157,  0.9160],
        [ 2.5014, -2.5060],
        [-2.2909,  2.0730],
        [ 0.7611, -0.7630],
        [ 1.6042, -1.6011],
        [ 2.4405, -2.3855],
        [ 1.6318, -1.6171],
        [ 0.5420, -0.5396],
        [ 2.9039, -2.9054],
        [-0.6316,  0.5629],
        [ 0.8953, -0.8821],
        [ 1.8593, -1.8574],
        [ 2.6072, -2.6002],
        [ 2.6847, -2.6828],
        [ 2.6218, -2.5971],
        [ 2.7206, -2.7376],
        [ 1.5615, -1

tensor([[-0.4904,  0.4315],
        [ 2.7293, -2.7215],
        [ 2.0048, -2.0068],
        [ 2.7886, -2.7723],
        [ 2.7548, -2.7891],
        [ 0.6395, -0.6646],
        [-2.4930,  2.2639],
        [-2.0555,  1.8827],
        [ 2.7644, -2.8185],
        [ 1.8387, -1.8674],
        [ 2.0188, -2.0014],
        [ 2.4354, -2.4823],
        [-1.9798,  1.7926],
        [ 1.8368, -1.8470],
        [ 1.4181, -1.4570],
        [-2.3060,  2.0957],
        [ 2.2724, -2.2987],
        [ 1.7718, -1.8148],
        [ 0.9234, -0.9455],
        [ 2.2990, -2.3047],
        [-1.9818,  1.7909],
        [-0.8679,  0.7241],
        [ 2.7666, -2.8050],
        [-2.1248,  1.9428],
        [ 2.7395, -2.7455],
        [ 2.3381, -2.3256],
        [ 0.4832, -0.4945],
        [ 1.8845, -1.8856],
        [ 0.5288, -0.5683],
        [ 0.1927, -0.1971],
        [-0.5037,  0.4349],
        [ 1.0267, -1.0513],
        [ 2.3005, -2.3028],
        [ 2.0848, -2.0965],
        [ 0.5044, -0.5080],
        [ 2.6659, -2

tensor([[-2.0119,  1.8210],
        [ 2.6659, -2.6773],
        [ 0.6793, -0.6733],
        [ 2.8597, -2.8924],
        [-1.1733,  1.0959],
        [-0.8571,  0.7443],
        [-2.7307,  2.4619],
        [ 2.9250, -2.9205],
        [ 0.3346, -0.3462],
        [ 0.8841, -0.8826],
        [ 1.9512, -1.9592],
        [-2.1983,  2.0087],
        [ 0.7662, -0.7404],
        [ 2.7120, -2.7522],
        [ 2.8125, -2.8292],
        [ 2.5036, -2.5508],
        [-1.2321,  1.0837],
        [ 2.8140, -2.8703],
        [-1.8033,  1.6189],
        [ 1.4082, -1.4249],
        [-2.5696,  2.3342],
        [ 0.3277, -0.3569],
        [-2.2897,  2.0523],
        [-0.5511,  0.5381],
        [ 2.2175, -2.1630],
        [ 2.0048, -1.9728],
        [-1.4894,  1.3540],
        [-2.0119,  1.8210],
        [-1.6775,  1.5412],
        [-2.0585,  1.8636],
        [ 1.2025, -1.2002],
        [-2.7071,  2.4426],
        [-0.0580,  0.0137],
        [-1.9326,  1.7478],
        [-0.0949,  0.0580],
        [-2.4319,  2

tensor([[ 0.1966, -0.2210],
        [-2.6139,  2.3656],
        [-2.5967,  2.3503],
        [-2.2394,  2.0242],
        [-2.5568,  2.3141],
        [-2.0119,  1.8210],
        [-2.6578,  2.4014],
        [-2.4132,  2.1822],
        [ 0.2271, -0.2493],
        [-1.8298,  1.6527],
        [-2.3582,  2.1321],
        [-2.0119,  1.8210],
        [-2.4731,  2.2369],
        [-2.8162,  2.5479],
        [ 0.0961, -0.1328],
        [-2.4276,  2.1972],
        [-2.2968,  2.0762],
        [-1.5042,  1.3365],
        [-1.9586,  1.7709],
        [-1.9805,  1.7942],
        [ 1.9533, -1.9611],
        [-1.8865,  1.6938],
        [-0.7823,  0.6938],
        [-0.1226,  0.0686],
        [ 2.8442, -2.8463],
        [-2.4502,  2.2166],
        [-2.6231,  2.3654],
        [-0.6683,  0.5544],
        [-2.0119,  1.8210],
        [-2.8389,  2.5728],
        [ 1.2225, -1.2442],
        [-2.2178,  2.0090],
        [-1.9326,  1.7478],
        [ 1.8632, -1.8347],
        [ 1.3402, -1.3203],
        [-1.7445,  1

...

tensor([[ 2.7917, -2.8357],
        [ 1.1262, -1.1290],
        [-1.7060,  1.5364],
        [ 2.8705, -2.9023],
        [ 1.8522, -1.8431],
        [ 1.7973, -1.8233],
        [-2.7387,  2.4674],
        [-2.8039,  2.5415],
        [ 1.0149, -0.9635],
        [ 0.5416, -0.5864],
        [-2.5298,  2.3011],
        [-1.4475,  1.3152],
        [ 2.2258, -2.2193],
        [-1.7060,  1.5364],
        [ 1.5723, -1.5842],
        [-2.8712,  2.5902],
        [ 2.5352, -2.5601],
        [ 2.7801, -2.8078],
        [ 1.5115, -1.5352],
        [-0.3545,  0.3168],
        [-2.3178,  2.1115],
        [ 2.8549, -2.8567],
        [-2.1670,  1.9227],
        [ 0.5029, -0.5200],
        [-2.2780,  2.0636],
        [ 2.3221, -2.3225],
        [ 1.3857, -1.3633],
        [-0.5525,  0.4891],
        [ 0.7036, -0.7180],
        [ 2.0363, -2.0129],
        [-0.0070, -0.0080],
        [ 2.0004, -2.0372],
        [-1.7424,  1.5733],
        [ 2.1483, -2.0933],
        [-0.6962,  0.6199],
        [-1.9425,  1

tensor([[ 1.9546, -1.9112],
        [ 2.4561, -2.5011],
        [ 1.5602, -1.5874],
        [ 2.0523, -2.0668],
        [ 1.4882, -1.4664],
        [ 2.8291, -2.8490],
        [ 1.3075, -1.3094],
        [ 0.6522, -0.6839],
        [ 2.7882, -2.7925],
        [ 2.7740, -2.8225],
        [ 0.9310, -0.9447],
        [-0.6174,  0.5469],
        [ 2.8215, -2.8539],
        [ 2.0137, -1.9884],
        [ 2.5676, -2.5480],
        [ 1.4144, -1.4472],
        [ 1.7656, -1.7705],
        [ 2.2963, -2.3052],
        [ 0.9277, -0.9087],
        [ 2.8040, -2.8110],
        [ 0.7152, -0.7340],
        [ 2.8053, -2.8184],
        [-2.5483,  2.3032],
        [ 2.6825, -2.7195],
        [ 2.6060, -2.6336],
        [ 2.6782, -2.6979],
        [ 2.1102, -2.1333],
        [ 2.1193, -2.1324],
        [ 0.8368, -0.8393],
        [ 2.8623, -2.8963],
        [ 2.9336, -2.9390],
        [-2.1376,  1.9533],
        [-1.9569,  1.7699],
        [ 0.1679, -0.1575],
        [-2.1717,  1.9514],
        [-0.9145,  0

tensor([[-2.5002,  2.2603],
        [-2.5568,  2.3126],
        [ 0.3975, -0.3541],
        [-1.7060,  1.5364],
        [ 2.6565, -2.6770],
        [-2.8341,  2.5564],
        [-2.6241,  2.3942],
        [-2.4082,  2.1698],
        [-2.8834,  2.6128],
        [-1.7050,  1.5311],
        [-2.8080,  2.5424],
        [-1.7060,  1.5364],
        [ 2.8669, -2.8780],
        [ 1.3036, -1.3190],
        [ 0.2382, -0.2719],
        [ 2.3457, -2.3764],
        [ 2.7997, -2.8299],
        [-2.0119,  1.8210],
        [ 1.5529, -1.5394],
        [-2.0119,  1.8210],
        [-1.8891,  1.7035],
        [-2.7498,  2.4927],
        [-2.1807,  1.9807],
        [ 1.0356, -0.9938],
        [-1.7060,  1.5364],
        [-1.8217,  1.6424],
        [-2.0117,  1.8263],
        [-2.0119,  1.8210],
        [-2.3045,  2.0837],
        [-1.7060,  1.5364],
        [ 2.8787, -2.8959],
        [ 2.7474, -2.7609],
        [-1.9537,  1.7633],
        [ 1.3068, -1.3168],
        [ 1.5199, -1.5776],
        [ 2.3090, -2

tensor([[ 2.6381, -2.6890],
        [-2.7341,  2.4570],
        [-1.4375,  1.3115],
        [ 2.4243, -2.4525],
        [-0.7430,  0.6650],
        [ 2.8072, -2.8454],
        [ 2.5989, -2.5989],
        [ 1.1750, -1.1391],
        [ 2.3696, -2.3765],
        [ 0.9188, -0.9433],
        [-2.0582,  1.8618],
        [-2.7425,  2.4744],
        [ 2.8880, -2.9020],
        [ 0.8080, -0.8066],
        [ 2.5700, -2.6143],
        [ 2.6412, -2.6699],
        [-0.1150,  0.0595],
        [ 1.6406, -1.6300],
        [ 2.1816, -2.1599],
        [ 2.7552, -2.7933],
        [ 2.3944, -2.3890],
        [ 0.1959, -0.2098],
        [ 2.5474, -2.5851],
        [-1.3711,  1.2472],
        [-0.0977,  0.0783],
        [ 2.8106, -2.8402],
        [ 2.2845, -2.3266],
        [ 2.8301, -2.8781],
        [ 1.3345, -1.3118],
        [ 0.6806, -0.7105],
        [ 2.8821, -2.8988],
        [ 2.7130, -2.7297],
        [ 2.9103, -2.9448],
        [ 2.6688, -2.7133],
        [ 1.8260, -1.7656],
        [ 2.6864, -2

tensor([[ 2.8111e+00, -2.8262e+00],
        [ 1.3902e+00, -1.4040e+00],
        [ 1.8653e+00, -1.9088e+00],
        [-2.4490e+00,  2.2150e+00],
        [ 2.2845e+00, -2.2719e+00],
        [-1.5274e+00,  1.3674e+00],
        [-9.5835e-04, -3.4853e-02],
        [-2.0119e+00,  1.8210e+00],
        [ 1.8619e+00, -1.8720e+00],
        [-2.4972e+00,  2.2497e+00],
        [ 2.4972e+00, -2.5227e+00],
        [ 2.7487e+00, -2.7905e+00],
        [-1.7910e+00,  1.5859e+00],
        [ 2.6404e+00, -2.6627e+00],
        [-2.3565e-01,  1.8273e-01],
        [ 2.7763e+00, -2.8170e+00],
        [ 1.1313e+00, -1.1401e+00],
        [ 2.0069e+00, -1.9571e+00],
        [ 2.5408e+00, -2.5804e+00],
        [ 2.2293e+00, -2.2665e+00],
        [ 1.4420e+00, -1.4708e+00],
        [ 1.8530e+00, -1.9140e+00],
        [ 3.2717e-01, -3.2220e-01],
        [ 2.7077e+00, -2.7001e+00],
        [-2.0313e+00,  1.8423e+00],
        [-2.5446e+00,  2.3245e+00],
        [ 6.5472e-01, -6.7861e-01],
        [ 2.1677e+00, -2.150

In [108]:
result

array([[0.9951894 , 0.00481058],
       [0.18345745, 0.81654257],
       [0.31010544, 0.6898945 ],
       ...,
       [0.00727459, 0.9927254 ],
       [0.00532816, 0.9946719 ],
       [0.5334392 , 0.46656075]], dtype=float32)

In [109]:
result_list = []

for i in result:
    result_list.append(i[1])

In [110]:
result_list

[0.004810582,
 0.81654257,
 0.6898945,
 0.43120867,
 0.97885203,
 0.9906262,
 0.99092966,
 0.037172735,
 0.99328184,
 0.005128247,
 0.09091287,
 0.0077646053,
 0.005327241,
 0.99389863,
 0.008693327,
 0.97777224,
 0.012678146,
 0.005409829,
 0.0062242527,
 0.12026703,
 0.0046129255,
 0.0038688525,
 0.9795592,
 0.9792515,
 0.0036213002,
 0.9873202,
 0.004689557,
 0.16336344,
 0.84377664,
 0.09756534,
 0.0053621163,
 0.060906164,
 0.0073556663,
 0.008634005,
 0.0034842386,
 0.071911946,
 0.088799186,
 0.9190736,
 0.005287886,
 0.18417373,
 0.11507185,
 0.003568735,
 0.2931962,
 0.74212456,
 0.0034421037,
 0.9644956,
 0.0033074012,
 0.0050986325,
 0.25416374,
 0.023002423,
 0.026767816,
 0.8432782,
 0.0061894045,
 0.96239793,
 0.034997907,
 0.7054591,
 0.58709073,
 0.0030906908,
 0.0052998606,
 0.0069923406,
 0.010665384,
 0.98579794,
 0.9853007,
 0.96934503,
 0.97881055,
 0.54707724,
 0.968561,
 0.91808784,
 0.99367225,
 0.0066211917,
 0.26176202,
 0.010438539,
 0.73333174,
 0.99512297,


In [111]:
tmp_df = pd.DataFrame(lines)

In [112]:
tmp_df = pd.DataFrame(list(tmp_df.applymap(lambda a: a.split('\t'))[0]))

In [113]:
tmp_df = tmp_df.rename(columns=tmp_df.iloc[0]).iloc[1:].reset_index(drop=True)

In [114]:
tmp_df['NSMC'] = result_list

In [121]:
tmp_df['score'] = tmp_df['score'].astype(int)

In [126]:
tmp_df['MAE'] = tmp_df['score'] - tmp_df['NSMC']

In [132]:
tmp_df

,reviewId,content,score,NSMC,,MAE
0,4123d655-f7b7-4328-abd2-7d7dcfb1c21f,직관성 떨어지고 오류나고 답답,1,0.004811,0.995189,0.995189
1,39afd67f-746b-462b-a838-f8dd5efca23e,이 부분 개선되면 참 좋겠습니다,1,0.816543,0.183457,0.183457
2,01016f38-580c-4c64-a954-5abf1f796094,불법이 아니라면 개인투자자의 투자자율성 편의를 강화하는게 앱의 활용성을 높이고 경쟁...,1,0.689894,0.310106,0.310106
3,2630586a-03c8-41bb-ac4e-04a961b0e01b,칸이모자르면 차라리 등락금액을 빼고 거래량을 넣어주세요,0,0.431209,-0.431209,-0.431209
4,c124f0ff-c50a-47dc-a971-d5cf2cb09696,좋아요..👌.,1,0.978852,0.021148,0.021148
...,...,...,...,...,...,...
4427,4c5a5486-9f87-4571-a97e-ba5353cddccc,아쉬운건 수수료가 비싸다는 점..,1,0.984253,0.015747,0.015747
4428,9100e8a6-7d71-4ea7-a972-145d5c1d1821,상당한 발전이군요,1,0.992318,0.007682,0.007682
4429,e620e368-649a-4d0d-a772-c4cab76470e1,시세알림 기능도 유용합니다.,1,0.992725,0.007275,0.007275
4430,471a8ecd-a646-4435-8246-3c924edd647b,로그인없이 시세 조회도 가능하고 일단 속도가 다른 어플에 비해서 빠르네요^^,1,0.994672,0.005328,0.005328


In [130]:
np.mean(abs(tmp_df['MAE']))

0.17955200966715096

In [86]:
train_df.to_csv('finetuning_fault.csv', index=False, encoding='utf-8-sig')

In [18]:
tmp = infer(train_df['content'][0], '/home/hjgp/research/KoELECTRA/finetune/ckpt/koelectra-small-v3-nsmc-ckpt/checkpoint-46000/')

AttributeError: 'ElectraTokenizer' object has no attribute 'input_ids'

In [ ]:
tmp[0]

In [215]:
result

array([[9.9700373e-01, 2.9962957e-03],
       [9.9898189e-01, 1.0181338e-03],
       [9.5161390e-01, 4.8386134e-02],
       ...,
       [9.9904662e-01, 9.5336186e-04],
       [8.5410187e-03, 9.9145901e-01],
       [9.9907494e-01, 9.2508737e-04]], dtype=float32)

In [168]:
train_df['score'][:9]

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
Name: score, dtype: int64

In [36]:
 1

'이거 어플 왜이렇게 멈춤현상 렉걸림이ㅡ심한가요'

In [22]:
train_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,date
0,a74ab9e7-5b6b-4665-b8e5-4c0a82459cf6,김선구,https://play-lh.googleusercontent.com/a/AItbvm...,"고객정보확인인가 그거 신분증 실패했다고 하거나, 본인확인증 찍고 다음화면이 잘려서 ...",3,0,8.90,2022-08-08 07:21:10,NaN,NaN,202208
1,0df5ae4e-ff08-434a-b7a2-62da3181b586,정수정,https://play-lh.googleusercontent.com/a/AItbvm...,카카오페이 인증으로 접속이 안되서 ID로 접속하려구 했는데 등록된 단말이 아니라고 ...,1,0,8.88,2022-08-07 23:38:18,NaN,NaN,202208
2,c3169719-6a19-4995-92f2-2eca597ef652,zenter,https://play-lh.googleusercontent.com/a-/AFdZu...,"PC용까지 쓰려면 믿고 걸러야 합니다. 600만원짜리 PC, 3개의 HTS 사용자지...",1,0,8.90,2022-08-05 23:52:17,NaN,NaN,202208
3,cda3efef-4429-4007-b1cb-14bd6fd42b4b,홍길동,https://play-lh.googleusercontent.com/a/AItbvm...,미장 주식 업데이트 할때마다 회사 주주정보에 총주식수와 유동주식수 나오던게 업데이트...,2,0,8.90,2022-08-05 23:22:37,NaN,NaN,202208
4,996572c2-f4cb-4efe-a400-40b0f0123413,이용현,https://play-lh.googleusercontent.com/a/AItbvm...,해외주식도 차트에 매매내역을 남길 수 있도록 해주세요! 꼭 부탁드려요!,5,0,8.90,2022-08-05 15:22:57,NaN,NaN,202208
...,...,...,...,...,...,...,...,...,...,...,...
5009,4c5a5486-9f87-4571-a97e-ba5353cddccc,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,초기 편집이랑 설정잘해놓으면 hts못지않네요 아쉬운건 수수료가 비싸다는 점..,5,0,1.15,2012-03-27 02:23:04,NaN,NaN,201203
5010,9100e8a6-7d71-4ea7-a972-145d5c1d1821,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,피씨버전을하는듯하네요 상당한 발전이군요,5,1,NaN,2012-03-16 18:39:06,NaN,NaN,201203
5011,e620e368-649a-4d0d-a772-c4cab76470e1,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,실행 아주 빠릅니다. 계정이 없어도 시세조회 가능합니다. 한번 로그인하면 별도의 로...,5,6,1.06,2011-11-10 20:40:07,NaN,NaN,201111
5012,471a8ecd-a646-4435-8246-3c924edd647b,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,로그인없이 시세 조회도 가능하고 일단 속도가 다른 어플에 비해서 빠르네요^^,5,2,1.05,2011-11-01 15:28:32,NaN,NaN,201111


In [46]:
ll = train_df.iloc[0][['reviewId', 'content', 'score']].to_string(header=False,index=False).strip()
ll

'a74ab9e7-5b6b-4665-b8e5-4c0a82459cf6\n고객정보확인인가 그거 신분증 실패했다고 하거나, 본인확인증 찍고 다음화면이 잘려서 수...\n                                                 3'

In [42]:
ll.split("\t")

['reviewId                 a74ab9e7-5b6b-4665-b8e5-4c0a82459cf6\ncontent     고객정보확인인가 그거 신분증 실패했다고 하거나, 본인확인증 찍고 다음화면이 잘려서 ...\nscore                                                       3\nName: 0, dtype: object']

In [32]:
train_df.iloc[1]

reviewId                             0df5ae4e-ff08-434a-b7a2-62da3181b586
userName                                                              정수정
userImage               https://play-lh.googleusercontent.com/a/AItbvm...
content                 카카오페이 인증으로 접속이 안되서 ID로 접속하려구 했는데 등록된 단말이 아니라고 ...
score                                                                   1
thumbsUpCount                                                           0
reviewCreatedVersion                                                 8.88
at                                                    2022-08-07 23:38:18
replyContent                                                          NaN
repliedAt                                                             NaN
date                                                               202208
Name: 1, dtype: object

In [33]:
for i in range(10):
    print(infer(train_df['content'][:10][i],'/home/hjgp/research/KoELECTRA/finetune/ckpt/koelectra-small-v3-nsmc-ckpt/checkpoint-46000/'))

tensor([[0.1503, 0.8497]], grad_fn=<SoftmaxBackward0>)
tensor([[0.9921, 0.0079]], grad_fn=<SoftmaxBackward0>)
tensor([[9.9965e-01, 3.5184e-04]], grad_fn=<SoftmaxBackward0>)
tensor([[0.9986, 0.0014]], grad_fn=<SoftmaxBackward0>)
tensor([[0.0178, 0.9822]], grad_fn=<SoftmaxBackward0>)
tensor([[9.9952e-01, 4.8041e-04]], grad_fn=<SoftmaxBackward0>)
tensor([[9.9961e-01, 3.9457e-04]], grad_fn=<SoftmaxBackward0>)
tensor([[0.0081, 0.9919]], grad_fn=<SoftmaxBackward0>)
tensor([[0.9989, 0.0011]], grad_fn=<SoftmaxBackward0>)
tensor([[0.9989, 0.0011]], grad_fn=<SoftmaxBackward0>)
